In [ ]:
# Copyright 2026 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Vertex AI Model Garden - PaliGemma 2 (Deployment)

<table><tbody><tr>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/notebooks/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/community/model_garden/model_garden_hf_paligemma2_deployment.ipynb">
      <img alt="Workbench logo" src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" width="32px"><br> Run in Workbench
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https:%2F%2Fraw.githubusercontent.com%2FGoogleCloudPlatform%2Fvertex-ai-samples%2Fmain%2Fnotebooks%2Fcommunity%2Fmodel_garden%2Fmodel_garden_hf_paligemma2_deployment.ipynb">
      <img alt="Google Cloud Colab Enterprise logo" src="https://lh3.googleusercontent.com/JmcxdQi-qOpctIvWKgPtrzZdJJK-J3sWE1RsfjZNwshCFgE_9fULcNpuXYTilIR2hjwN" width="32px"><br> Run in Colab Enterprise
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/model_garden/model_garden_hf_paligemma2_deployment.ipynb">
      <img alt="GitHub logo" src="https://github.githubassets.com/assets/GitHub-Mark-ea2971cee799.png" width="32px"><br> View on GitHub
    </a>
  </td>
</tr></tbody></table>

## Overview

This notebook demonstrates how to deploy a **Paligemma** open model on Google Cloud Vertex AI.

### Objectives

- Deploy Paligemma using containerized backends like [vLLM](https://github.com/vllm-project/vllm) on GPU.
- Use the deployed model to serve chat completion requests for both text and multimodal inputs.

### File a Bug

If you encounter issues with this notebook, report them on [GitHub](https://github.com/GoogleCloudPlatform/vertex-ai-samples/issues/new).

### Costs

This tutorial uses billable components of Google Cloud:

- Vertex AI
- Cloud Storage

Refer to the [Vertex AI pricing](https://cloud.google.com/vertex-ai/pricing) and [Cloud Storage pricing](https://cloud.google.com/storage/pricing) pages for more information. Use the [Pricing Calculator](https://cloud.google.com/products/calculator/) to estimate your projected costs.

## Get Started

### Install Vertex AI SDK and other required packages

In [ ]:
%pip install --upgrade --force-reinstall --quiet 'google-cloud-aiplatform>=1.106.0' 'openai' 'google-auth==2.27.0' 'requests==2.32.3'

### Authenticate the Notebook Environment (Colab only)

If you're running this notebook in Google Colab, run the following cell to authenticate.

In [ ]:
import sys

if "google.colab" in sys.modules:
    from google.colab import auth

    auth.authenticate_user()

### Set Google Cloud Project Information

To get started with Vertex AI, ensure you have an existing Google Cloud project and that the [Vertex AI API is enabled](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com).

See the guide on [setting up your project and development environment](https://cloud.google.com/vertex-ai/docs/start/cloud-environment). Also confirm that [billing is enabled](https://cloud.google.com/billing/docs/how-to/modify-project).


In [ ]:
# Use the environment variable if the user doesn't provide Project ID.
import os

import vertexai

PROJECT_ID = ""  # @param {type: "string", placeholder: "[your-project-id]", isTemplate: true}

if not PROJECT_ID:
    PROJECT_ID = os.environ.get("GOOGLE_CLOUD_PROJECT")

REGION = ""  # @param {type: "string", placeholder: "[your-region]", isTemplate: true}

if not REGION:
    REGION = os.environ.get("GOOGLE_CLOUD_REGION", "us-central1")

vertexai.init(project=PROJECT_ID, location=REGION)

print(f"Project: {PROJECT_ID}\nLocation: {REGION}")

### Import libraries

In [ ]:
from vertexai import model_garden

## Deploy model

### Choose model variant

You can proceed with the default model variant or select a different one.

In [ ]:
model_version = "paligemma2-3b-mix-448"  # @param ["paligemma-224-bfloat16", "paligemma-224-float16", "paligemma-224-float32", "paligemma-448-bfloat16", "paligemma-448-float16", "paligemma-448-float32", "paligemma-896-bfloat16", "paligemma-896-float16", "paligemma-896-float32", "paligemma-mix-224-bfloat16", "paligemma-mix-224-float16", "paligemma-mix-224-float32", "paligemma-mix-448-bfloat16", "paligemma-mix-448-float16", "paligemma-mix-448-float32", "paligemma2-10b-ft-docci-448", "paligemma2-10b-mix-224", "paligemma2-10b-mix-448", "paligemma2-10b-pt-448", "paligemma2-10b-pt-896", "paligemma2-28b-mix-224", "paligemma2-28b-mix-448", "paligemma2-28b-pt-224", "paligemma2-28b-pt-448", "paligemma2-28b-pt-896", "paligemma2-3b-ft-docci-448", "paligemma2-3b-mix-224", "paligemma2-3b-mix-448", "paligemma2-3b-pt-224", "paligemma2-3b-pt-448", "paligemma2-3b-pt-896"] {isTemplate:true}
MODEL_NAME = f"google/paligemma@{model_version}"

To see all deployable model variants available in Model Garden, use:

In [ ]:
all_model_versions = model_garden.list_deployable_models(
    model_filter="paligemma", list_hf_models=False
)

Once you've selected a model variant, initialize it:

In [ ]:
model = model_garden.OpenModel(MODEL_NAME)

### Check the Deployment Configuration

Use the `list_deploy_options()` method to view the verified deployment configurations for your selected model. This helps ensure you have sufficient resources (e.g., GPU quota) available to deploy it.

In [ ]:
deploy_options = model.list_deploy_options(concise=True)
print(deploy_options)

### Deploy the Model

Now that you’ve reviewed the deployment options, use the `deploy()` method to serve the selected open model to a Vertex AI endpoint. Deployment time may vary depending on the model size and infrastructure requirements.

> **Note**: If the model requires accepting a license agreement (EULA), set the `accept_eula=True` flag in the deploy call. Set `use_dedicated_endpoint` to False if you don't want to use [dedicated endpoint](https://cloud.google.com/vertex-ai/docs/general/deployment#create-dedicated-endpoint).

In [ ]:
use_dedicated_endpoint = True

In [ ]:
endpoints = {}

In [ ]:
endpoints["sdk_default"] = model.deploy(
    accept_eula=True,
    use_dedicated_endpoint=use_dedicated_endpoint,
)

Alternatively, you can select one of the verified deployment configurations listed above.

In [ ]:
endpoints["sdk_custom"] = model.deploy(
    accept_eula=True,
    use_dedicated_endpoint=use_dedicated_endpoint,
    serving_container_image_uri="us-docker.pkg.dev/vertex-ai/vertex-vision-model-garden-dockers/pytorch-one-serve:20250205_0822_RC00",
    machine_type="g2-standard-16",
    accelerator_type="NVIDIA_L4",
    accelerator_count=1,
)

In [ ]:
if "sdk_default" in endpoints:
    endpoint = endpoints["sdk_default"]
    LABEL = "sdk_default"
elif "sdk_custom" in endpoints:
    endpoint = endpoints["sdk_custom"]
    LABEL = "sdk_custom"
else:
    raise ValueError("No endpoint found. Create an endpoint.")

To further customize your deployment, you can configure:

- **Compute Resources**: Machine type, replica count (min/max), accelerator type and quantity.
- **Infrastructure**: Use Spot VMs, reservation affinity, or dedicated endpoints.
- **Serving Container**: Customize container image, ports, health checks, and environment variables.

See the [Model Garden SDK README](https://github.com/googleapis/python-aiplatform/blob/main/vertexai/model_garden/README.md) for advanced configuration options.

## Predict

The following sections will use images from [pexels.com](https://www.pexels.com/) for demoing purposes. All the images have the following license: https://www.pexels.com/license/.

Images will be resized to a width of 1000 pixels by default since requests made to a Vertex Endpoint are limited to 1.500MB.

In [ ]:
# @title Visual Question Answering

# @markdown This section uses the deployed PaliGemma model to answer questions about a given image.

import importlib
from typing import Any, Dict

from google.cloud import aiplatform

# Import the necessary packages.
! rm -rf vertex-ai-samples && git clone https://github.com/GoogleCloudPlatform/vertex-ai-samples.git
! cd vertex-ai-samples

common_util = importlib.import_module(
    "vertex-ai-samples.notebooks.community.model_garden.docker_source_codes.notebook_util.common_util"
)

# @markdown ![](https://images.pexels.com/photos/1006293/pexels-photo-1006293.jpeg?auto=compress&cs=tinysrgb&w=630&h=375&dpr=2)
image_url = "https://images.pexels.com/photos/1006293/pexels-photo-1006293.jpeg"  # @param {type:"string"}

# @markdown You may leave question prompts empty and they will be ignored.
question_prompt = "What is shown in the picture?"  # @param {type: "string"}

# @markdown The question prompt can be non-English languages.


def vqa_predict(
    endpoint: aiplatform.Endpoint,
    image_url: str,
    text_prompt: str,
    parameters: Dict[str, Any] = None,
) -> str:
    """Predicts the answer to a question about an image using an Endpoint,

       and passes parameters in the payload.

    Args:
        endpoint: The deployed Vertex AI endpoint.
        image_url: URL of the image to ask about.
        text_prompt: The text prompt question.
        parameters: Additional parameters for the prediction request.

    Returns:
        The predicted answer string or None if no prediction.
    """

    instances = []
    if text_prompt:
        instances.append(
            {
                "text_prompt": text_prompt,
                "image_url": image_url,
            }
        )

    # Construct the prediction payload
    payload = {"instances": instances}
    if parameters:
        payload["parameters"] = parameters

    response = endpoint.predict(instances=instances, parameters=parameters)
    answer = None
    if response.predictions:
        answer = response.predictions[0]["text"].split("\n")[1]
    return answer


# Using max_new_tokens along with other parameters
parameters_with_tokens = {"max_new_tokens": 50}
predictions_with_tokens = vqa_predict(
    endpoint=endpoint,
    image_url=image_url,
    text_prompt=question_prompt,
    parameters=parameters_with_tokens,
)


image = common_util.download_image(image_url)
display(image)

print(f"Prediction Response: {predictions_with_tokens}")
# @markdown Click "Show Code" to see more details.

## Clean up resources

In [ ]:
# @markdown Delete the endpoint.

for endpoint in endpoints.values():
    endpoint.delete(force=True)